# Optical mark recognition

### Importing the libraries

In [1]:
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import imutils
import cv2

### Initializing the answer key

In [2]:
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}
ANSWER_KEY_FAKE = 3;

### Reading the image

In [3]:
image = cv2.imread('data/Sheet1.jpg')

### Converting it to grayscale

In [4]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

### Bluring it to hide the details

In [5]:
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

### Performing edge detection

In [6]:
edged = cv2.Canny(blurred, 75, 200)

### Displaying the image

In [7]:
# cv2.imshow('img',edged)
# cv2.waitKey(0)

### Finding the contours

In [8]:
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

In [9]:
cnts = cnts[0]

### Determining contour of the paper

In [10]:
docCnt = None
if len(cnts) > 0:
     # Sorting contours based on their area. Paper will have mamimum area as it is the main object
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)  
    
    for c in cnts:
        
        peri = cv2.arcLength(c, True)   # Calculating the perimeter
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)    # approximating the shape
        
        if len(approx) == 4:    # If shape has four edges
            docCnt = approx
            break

### Visualizing the contours

In [11]:
# cv2.imshow('temp',cv2.drawContours(image.copy(),[docCnt],-1,(0,0,255),2))
# cv2.waitKey(0)

### Applying four point transformation

In [12]:
paper = four_point_transform(image,docCnt.reshape(4,2))

In [13]:
warped = four_point_transform(gray,docCnt.reshape(4,2))

### Visualizing the transformed image

In [14]:
y=160
x=60
h=200
w=220
crop_id = warped[y:y+h, x:x+w]
# cv2.imshow("cropped_id", crop_id)
# cv2.waitKey(0)

y=360
x=60
h=460
w=520
crop_answer = warped[y:y+h, x:x+w]
crop_answer_color = paper[y:y+h, x:x+w]
# cv2.imshow("cropped", crop_answer)
# cv2.waitKey(0)
# cv2.imshow('img',paper)
# cv2.waitKey(0)

### Thresholding

In [15]:
# OTSU will automatically determine the best value for the parameter thresh. The value of thresh is not considered 
# when otsu flag is passed
thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1] 

thresh_id = cv2.threshold(crop_id, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1] 
thresh_answer = cv2.threshold(crop_answer, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1] 
cv2.imshow("Thresh", thresh_answer)                                    
cv2.waitKey(0)

-1

### Finding the contours of the thresholded image

In [16]:
cnts = cv2.findContours(thresh_id.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts_id = cnts[0]

cnts = cv2.findContours(thresh_answer.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts_answer = cnts[0]

# cv2.imshow('temp',cv2.drawContours(pap,cnts,-1,(0,0,255),2))
# cv2.waitKey(0)

### Determining the contours of the bubble

In [17]:
idCnts = []
questionCnts = []
# for c in cnts:
for c in cnts_id:
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    
    if w <= 22 and w >= 15 and h <= 22 and h >= 15 and ar >= 0.7 and ar <= 1.3:
        idCnts.append(c)
        
for c in cnts_answer:
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    
    if w <= 22 and w >= 15 and h <= 22 and h >= 15 and ar >= 0.7 and ar <= 1.3:
        questionCnts.append(c) 

In [18]:
len(questionCnts)

320

### Visualizing the contour of bubbles

In [19]:
# pap = paper.copy()
# for c in questionCnts:
# cv2.imshow('temp',cv2.drawContours(pap,questionCnts,-1,(0,0,255),2))
# cv2.waitKey(0)

### Sorting the contours from top to bottom

In [20]:
idCnts = contours.sort_contours(idCnts, method="left-to-right")[0]
# questionCnts = contours.sort_contours(questionCnts, method="top-to-bottom")[0]
questionCnts = contours.sort_contours(questionCnts, method="left-to-right")[0]
# questionCnts = contours.sort_contours(questionCnts, method="top-to-bottom")[0]

### Visualizing the sorted contour of bubbles

In [21]:
# pap = paper.copy()
# for c in questionCnts:
#     cv2.imshow('temp',cv2.drawContours(pap,[c],-1,(0,255,0),2))
#     cv2.waitKey(0)

# pap = crop_id.copy()
# for c in idCnts:
#     cv2.imshow('temp',cv2.drawContours(pap, [c],-1,(0,255,0),2))
#     cv2.waitKey(0)

# pap = crop_answer.copy()
# for c in questionCnts:
#     cv2.imshow('temp',cv2.drawContours(pap, [c],-1,(0,255,0),2))
#     cv2.waitKey(0)

### Isolating the bubbles

In [22]:
pap = paper.copy()
studentId = ""
flag = False
for i in range(0,len(idCnts),10):
    temp = contours.sort_contours(idCnts[i:i+10], method="top-to-bottom")[0]
    
    bubbled = None
    flag = False
        
    for (j,c) in enumerate(temp):
        # Refer to last section ('Isolation step by step') for viewing what each step does 
        mask = np.zeros(thresh_id.shape, dtype="uint8")     # Tô đen
        cv2.drawContours(mask, [c], -1, 255, -1)         # Tô trắng                                             
        
        mask = cv2.bitwise_and(thresh_id, thresh_id, mask=mask)     # Doing Bitwise-and to reveal the bubble
        total = cv2.countNonZero(mask)     # Đếm trắng
        
        if total > 200:     # Chỉnh cái này
            bubbled = (total, j)
        
#     print(bubbled)
    if bubbled:
        studentId += str(bubbled[1])
        
print(studentId)

27860000


In [23]:
pap = crop_answer_color.copy()
q = -1
correct = 0
flag = False
# minm = []
# maxm = []
for i in range(0,len(questionCnts),80): #chỉnh số câu
    
    # Sorting contours from left to right
    temp = contours.sort_contours(questionCnts[i:i+80], method="left-to-right")[0] # chon cot 20 cau
    temp2 = contours.sort_contours(temp, method="top-to-bottom")[0]
    
    for i in range(0,len(temp2), 4):
        q+=1
        temp3 = contours.sort_contours(temp2[i:i+4], method="left-to-right")[0]
#         for c in temp3:
#             cv2.imshow('temp',cv2.drawContours(pap, [c],-1,(0,255,0),2))
#             cv2.waitKey(0)
    
        bubbled = None
        flag = False
        for (j,c) in enumerate(temp3):
            mask = np.zeros(thresh_answer.shape, dtype="uint8")     # Create a dummy mask
            cv2.drawContours(mask, [c], -1, 255, -1)         # Drawing the contours on the dummy mask
            
#             cv2.imshow('temp', mask)
#             cv2.waitKey(0)

            mask = cv2.bitwise_and(thresh_answer, thresh_answer, mask=mask)     # Doing Bitwise-and to reveal the bubble
            total = cv2.countNonZero(mask)     # Calculating the sum of non-zero pixels            

            
            # Breaking the loop and setting bubbled = None when another marked bubble is found in same question
            if flag and total > 150:     
                bubbled = None
                break

            if total > 150:     # Setting flag = True when a marked bubble is found
                flag = True

            if total > 150:     # Comparing the sum of non zero pixels
                bubbled = (total, j)
        
        color = (0, 0, 255)
        # k = ANSWER_KEY[q]     # Retrieving the answer from the answer key based on question 'q'
        k = 2 # default dap an la C
#         {15: 3}; {3}
        if bubbled and k == bubbled[1]:
            color = (0, 255, 0)
            correct += 1
        if bubbled:     # Not Drawing contour when bubbled is None
            cv2.drawContours(pap, [temp3[k]], -1, color, 3)

### Displaying the score

In [26]:
score = (correct /(q+1)) * 100
result = pap.copy()
cv2.putText(result, studentId, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
cv2.putText(result, "{:.1f}%".format(score), (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
# cv2.imshow("Original", image)
cv2.imshow("test", result)
cv2.waitKey(0)

-1